In [2]:
import numpy as np

In [3]:
et_path = '/home/infres/ypeng-21/work/Taxon_clean/build_dataset/test/build_data_from_nt/24_03_22/raw_data/dataset/Wiki_ET.txt'

In [4]:
mapped_wiki_ents_en = set()
with open('../wikipedia/enwiki', 'r') as enwiki:
    for line in enwiki:
        qid = line.strip().split(',')[1]
        mapped_wiki_ents_en.add('wd:'+str(qid))

In [4]:
len(mapped_wiki_ents_en)

9784084

In [31]:
with open('Wiki_ET_subset.txt', 'w') as etwriter: # after enwiki filtering
    with open(et_path, 'r') as et:
        for line in et:
            inst = line.strip().split('\t')[0]
            if inst in mapped_wiki_ents_en:
                etwriter.write(line)

In [15]:
# inst_stats = utils.subj_mentions('Wiki_ET_after_wikipedia_filtering.txt')
# len(inst_stats)

file_name: Wiki_ET_after_wikipedia_filtering.txt, file_size: 315975909, chunk_size: 2468561


6995458

* load ori / clean graph

In [5]:
from collections import defaultdict
import networkx as nx
ori_graph = '/home/infres/ypeng-21/work/Taxon_clean/build_dataset/test/build_data_from_nt/24_03_22/raw_data/wikidata_src/wiki_taxonomy.tsv'
clean_graph = '/home/infres/ypeng-21/work/Taxon_clean/LLM/data/graph/clean_wikiTaxonUp.tsv'

oriWikiTaxonDown = defaultdict(set)
with open(ori_graph, 'r') as ori:
    for line in ori:
        triple = line.strip().split('\t')
        if len(triple) > 3:
            oriWikiTaxonDown[triple[2]].add(triple[0])
oriWiki = nx.DiGraph(oriWikiTaxonDown)

cleanWikiTaxonDown = defaultdict(set)
with open(clean_graph, 'r') as clean:
    for line in clean:
        child, parent = line.strip().split('\t')
        cleanWikiTaxonDown[parent].add(child)
cleanWiki = nx.DiGraph(cleanWikiTaxonDown)

In [6]:
print('Original graph:', oriWiki.number_of_nodes(), oriWiki.number_of_edges())
print('Cleaned graph:', cleanWiki.number_of_nodes(), cleanWiki.number_of_edges())

Original graph: 3962959 4657644
Cleaned graph: 16613 19654


In [7]:
# redundant taxonomic links: oriwiki
oriWiki.number_of_edges() - nx.transitive_reduction(oriWiki).number_of_edges()

497890

* avg path to root

In [6]:
from tqdm import tqdm
cls_stats = defaultdict(int)
inst_cls_stats = defaultdict(int)
inst_cls = defaultdict(set)

path1 = '/home/infres/ypeng-21/work/Taxon_clean/build_dataset/test/build_data_from_nt/24_03_22/raw_data/inst_P31_cls.txt'
path2 = '/home/infres/ypeng-21/work/Taxon_clean/build_dataset/test/build_data_from_nt/24_03_22/raw_data/occupation_P106.txt'

with open(path1, 'r') as file:
    for line in tqdm(file, total=111397057):
        tupl = line.strip().split('\t')
        if len(tupl) < 2:
            continue
        else:
            inst, cls = tupl
            cls_stats[cls] += 1
            inst_cls_stats[inst] += 1
            inst_cls['wd:'+inst].add('wd:'+cls)

with open(path2, 'r') as file:
    for line in tqdm(file, total=11128995):
        tupl = line.strip().split('\t')
        if len(tupl) < 2:
            continue
        else:
            inst, cls = tupl
            cls_stats[cls] += 1
            inst_cls_stats[inst] += 1
            inst_cls['wd:'+inst].add('wd:'+cls)

100%|██████████| 11128995/11128995 [00:22<00:00, 485081.38it/s]


In [19]:
# retyping
tmp = np.load('edges_del.npy')
del_edges = set()  # tuple set
for edge in tmp:
    del_edges.add(tuple(edge))
print(len(del_edges)) 
wikiTaxonUp = nx.to_dict_of_lists(cleanWiki.reverse())


# Retyping
from itertools import permutations
global del_edges
def get_ancestors(digraph, ancestors, cls, depth):
    for sp in digraph.predecessors(cls):
        # path should not include irrelevant edges
        if tuple([sp, cls]) in del_edges:
            continue
        if sp in wikiTaxonUp:
            ancestors.add(tuple([sp, depth]))
            continue
        get_ancestors(digraph, ancestors, sp, depth+1)

def get_valid_first_ancestors(digraph, cls):
    ancestors = set()
    get_ancestors(digraph, ancestors, cls, depth=0)
    return ancestors


# Process Retype
inst2type = defaultdict(set)
for inst in tqdm(inst_cls.keys()):
    cls_set = inst_cls[inst]
    for cls in cls_set:
        if cls in wikiTaxonUp:
            # class avaliable in the clean graph
            inst2type[inst].add(cls)
            continue
        if not oriWiki.has_node(cls):
            continue
        # No direct class exists anymore
        fir_ancestors = list(get_valid_first_ancestors(oriWiki, cls))
        sorted_ances = sorted(fir_ancestors, key=lambda x: x[1])
        min_depth = 1e9
        # retype to the first ancestor
        for tupl in sorted_ances:
            ances, depth = tupl
            if depth <= min_depth:
                min_depth = depth
                inst2type[inst].add(ances)
            else:
                break


# remove transtive parents types for each instance
for inst in tqdm(inst2type.keys()):
    if len(inst2type[inst]) == 1:
        continue
    valcls = inst2type[inst].copy()
    for (node1,node2) in permutations(list(inst2type[inst]), 2):
        if node1 not in valcls or node2 not in valcls:
            continue
        if node1 == 'wd:Q35120': # not retyped to 'entity'
            valcls.remove(node1)
            continue
        if nx.has_path(oriWiki, node1, node2): # !!!!clean or Ori?? 不用clean的原因为，retyped到的很高层次的type很可能是错的
            valcls.remove(node1)
    inst2type[inst] = valcls.copy() # set


# filtering out instances typed to class of top3lavel classes
top3_cls = set(nx.single_source_shortest_path_length(cleanWiki, source='wd:Q35120', cutoff=3).keys())
print(len(top3_cls))

for inst in tqdm(inst2type.keys()):
    del_set = inst2type[inst].intersection(top3_cls)
    if len(del_set) > 0:
        for cls in del_set:
            inst2type[inst].remove(cls)


# check if empty set exists
print(len(inst2type))
clean_inst2type = {}
for inst in tqdm(inst2type.keys()):
    if len(inst2type[inst]) > 0:
        clean_inst2type[inst] = inst2type[inst].copy()
# delete inst2type
del inst2type
print(len(clean_inst2type))


10695


100%|██████████| 104277315/104277315 [8:58:10<00:00, 3229.36it/s]  


675


100%|██████████| 104277315/104277315 [01:58<00:00, 878049.76it/s]


104277315


100%|██████████| 104277315/104277315 [08:44<00:00, 198848.74it/s]


90580083


In [20]:
from tqdm import tqdm
def count_paths_to_root(G, root, nodes_list):
    path_counts = {}
    for node in tqdm(nodes_list):
        if node == root:
            path_counts[node] = 0  # Only one path from root to itself
        else:
            paths = list(nx.all_simple_paths(G, node, root))
            path_counts[node] = len(paths)
    return path_counts

In [21]:
new_cls_stats = defaultdict(int)
for inst in tqdm(clean_inst2type.keys()):
    for cls in clean_inst2type[inst]:
        new_cls_stats[cls] += 1

 68%|██████▊   | 61674934/90580083 [00:55<00:26, 1101647.48it/s]

100%|██████████| 90580083/90580083 [01:22<00:00, 1096317.61it/s]


In [22]:
root = 'wd:Q35120'
cls_list = list(new_cls_stats.keys())
path_counts = count_paths_to_root(cleanWiki.reverse(), root, cls_list)

total_inst_path_count = 0
for cls, count in path_counts.items():
    total_inst_path_count += count * new_cls_stats[cls]

average_paths = total_inst_path_count / sum(new_cls_stats.values())
print(f"Average number of paths to the root {root}: {average_paths:.2f}")

# 2.56 if don't remove transitive links
# 2.85 if remove transitive links

100%|██████████| 15881/15881 [00:00<00:00, 33062.60it/s]

Average number of paths to the root wd:Q35120: 2.85


In [10]:
# Classes without cumulative instances: oriwikis, strict as we set a lot of constraints on instances
path = '/home/infres/ypeng-21/work/Taxon_clean/build_dataset/test/build_data_from_nt/24_03_22/raw_data/dataset/cum_cls_inst_stats.txt'
cls_with_insts = set()
with open(path, 'r') as file:
    for line in file:
        cls, num = line.strip().split('\t')
        cls_with_insts.add(cls)
print("Classes without cumulative instances", oriWiki.number_of_nodes() - len(cls_with_insts))

Classes without cumulative instances 3880217


In [5]:
print("Is DAG?", nx.is_directed_acyclic_graph(oriWiki))
print("Is connected?", nx.is_weakly_connected(oriWiki))
print("Max Depth:", max(nx.shortest_path_length(oriWiki, source='wd:Q35120').values()))
print("Top-level classes", oriWiki.out_degree('wd:Q35120'))

Is DAG? True
Is connected? True
Max Depth: 19
Top-level classes 38


In [6]:
print("Is DAG?", nx.is_directed_acyclic_graph(cleanWiki))
print("Is connected?", nx.is_weakly_connected(cleanWiki))
print("Max Depth:", max(nx.shortest_path_length(cleanWiki, source='wd:Q35120').values()))
print("Top-level classes", cleanWiki.out_degree('wd:Q35120'))

Is DAG? True
Is connected? True
Max Depth: 13
Top-level classes 16


In [7]:
import Prefixes
import numpy as np
import pandas as pd
import multiprocessing as mp
import os


def prop_mentions(file_path):
    return parallel_read(file_path, count_properties)


def ent_mentions(file_path):
    return parallel_read(file_path, count_entities)


def cls_mentions(file_path):
    return parallel_read(file_path, count_class_insts)


def inst_type_mentions(file_path):
    return parallel_read(file_path, count_inst_types)


def subj_mentions(file_path):
    return parallel_read(file_path, count_subj)


def process_chunk(file_name, chunk_start, chunk_end, func):
    # chunk_results = []
    with open(file_name, 'r', encoding='UTF-8') as f:
        # Moving stream position to `chunk_start`
        f.seek(chunk_start)

        # Read and process lines until `chunk_end`
        chunk_results = defaultdict(int)
        for line in f:
            chunk_start += len(line)
            if chunk_start > chunk_end:
                break

            elms = func(line)
            if len(elms) > 0:
                for e in elms:
                    chunk_results[e] += 1
    return chunk_results


def parallel_read(file_name, func):
    # Maximum number of processes we can run at a time
    cpu_count = mp.cpu_count()

    file_size = os.path.getsize(file_name)
    chunk_size = np.floor(file_size / cpu_count).astype(int)
    print(f'file_name: {file_name}, file_size: {file_size}, chunk_size: {chunk_size}')

    # Arguments for each chunk (eg. [('input.txt', 0, 32), ('input.txt', 32, 64)])
    chunk_args = []
    with open(file_name, 'rb') as f: # 'rb' avoding utf-8 decoding problems
        # def is_start_of_line(position):
        #     if position == 0:
        #         return True
        #     # Check whether the previous character is EOL
        #     f.seek(position - 1)
        #     return f.read(1) == '\n'

        def get_next_line_position(position):
            # Read the current line till the end
            f.seek(position)
            f.readline()
            # Return a position after reading the line
            return f.tell()

        chunk_start = 0
        # Iterate over all chunks and construct arguments for `process_chunk`
        while chunk_start < file_size:
            chunk_end = min(file_size, chunk_start + chunk_size)

            # # Make sure the chunk ends at the beginning of the next line
            # while not is_start_of_line(chunk_end):
            #     chunk_end -= 1

            # Handle the case when a line is too long to fit the chunk size
            # if chunk_start == chunk_end:
            chunk_end = get_next_line_position(chunk_end)

            # Save `process_chunk` arguments
            args = (file_name, chunk_start, chunk_end, func)
            chunk_args.append(args)

            # Move to the next chunk
            chunk_start = chunk_end

    with mp.Pool(cpu_count) as p:
        # Run chunks in parallel
        chunk_results = p.starmap(process_chunk, chunk_args)

    results = defaultdict(int)
    # Combine chunk results into `results`
    for chunk_result in chunk_results:
        for r in chunk_result.keys():
            results[r] += chunk_result[r]
    return results


def count_properties(line):
    if not line.startswith("#") and not line.startswith("@"):
        triples = line.rstrip().split("\t")
        if len(triples)>2:
            return [triples[1]]
    return []


def count_entities(line):
    if not line.startswith("#") and not line.startswith("@"):
        triples = line.rstrip().split("\t")
        if len(triples)>2:
            return [triples[0], triples[2]]
    return []


def count_class_insts(line):
    if not line.startswith("#") and not line.startswith("@"):
        triples = line.rstrip().split("\t")
        if len(triples)>2 and triples[1] == Prefixes.rdfType:
            return [triples[2]] # class information
    return []


def count_inst_types(line):
    if not line.startswith("#") and not line.startswith("@"):
        triples = line.rstrip().split("\t")
        if len(triples)>2 and triples[1] == Prefixes.rdfType:
            return [triples[0]] # instance information
    return []


def count_subj(line):
    if not line.startswith("#") and not line.startswith("@"):
        triples = line.rstrip().split("\t")
        if len(triples)>2:
            return [triples[0]]
    return []

* load the instances

In [ ]:
from tqdm import tqdm
cls_stats = defaultdict(int)
path = '/home/infres/ypeng-21/work/Taxon_clean/build_dataset/test/build_data_from_nt/24_03_22/raw_data/dataset/Wiki_ET.txt'
with open(path, 'r') as file:
    for line in file:
        inst, rel, cls = line.strip().split('\t')
        cls_stats[cls] += 1

* Retyping the instances

In [8]:
# load data
tmp = np.load('edges_del.npy')
del_edges = set()  # tuple set
for edge in tmp:
    del_edges.add(tuple(edge))
print(len(del_edges)) 
wikiTaxonUp = nx.to_dict_of_lists(cleanWiki.reverse())

10695


In [9]:
# Retyping
from itertools import permutations
global del_edges
def get_ancestors(digraph, ancestors, cls, depth):
    for sp in digraph.predecessors(cls):
        # path should not include irrelevant edges
        if tuple([sp, cls]) in del_edges:
            continue
        if sp in wikiTaxonUp:
            ancestors.add(tuple([sp, depth]))
            continue
        get_ancestors(digraph, ancestors, sp, depth+1)

def get_valid_first_ancestors(digraph, cls):
    ancestors = set()
    get_ancestors(digraph, ancestors, cls, depth=0)
    return ancestors

In [10]:
if 'wd:Q35120' in wikiTaxonUp:
    print('wd:Q35120 in wikiTaxonUp')

wd:Q35120 in wikiTaxonUp


In [11]:
# Process Retype
from tqdm import tqdm
inst2type = defaultdict(set)
with open('Wiki_ET_subset.txt', 'r') as et_file:
    for line in et_file:
        inst, rel, cls = line.strip().split('\t')
        if cls in wikiTaxonUp:
            inst2type[inst].add(cls)
            continue
        # No direct class exists anymore
        fir_ancestors = list(get_valid_first_ancestors(oriWiki, cls))
        sorted_ances = sorted(fir_ancestors, key=lambda x: x[1])
        min_depth = 1e9
        for tupl in sorted_ances:
            ances, depth = tupl
            if depth <= min_depth:
                min_depth = depth
                inst2type[inst].add(ances)
            else:
                break

In [12]:
# some leaves classes doesn't have an instance as here we do wikipedia filtering, so some classes may not have instances anymore
if 'wd:Q85946972' in inst2type:
    print(inst2type['wd:Q85946972'])

In [13]:
# remove transtive parents types for each instance
for inst in tqdm(inst2type.keys()):
    if len(inst2type[inst]) == 1:
        continue
    valcls = inst2type[inst].copy()
    for (node1,node2) in permutations(list(inst2type[inst]), 2):
        if node1 not in valcls or node2 not in valcls:
            continue
        if node1 == 'wd:Q35120': # not retyped to 'entity'
            valcls.remove(node1)
            continue
        if nx.has_path(oriWiki, node1, node2): # !!!!clean or Ori?? 不用clean的原因为，retyped到的很高层次的type很可能是错的
            valcls.remove(node1)
    inst2type[inst] = valcls.copy() # set

  0%|          | 0/7204396 [00:00<?, ?it/s]

100%|██████████| 7204396/7204396 [06:10<00:00, 19447.81it/s] 


In [147]:
# check wd:Q1258, wd:Q884142
inst2type['wd:Q1258']

In [14]:
# filtering out instances typed to class of top3lavel classes
top3_cls = set(nx.single_source_shortest_path_length(cleanWiki, source='wd:Q35120', cutoff=3).keys())
print(len(top3_cls))

for inst in tqdm(inst2type.keys()):
    del_set = inst2type[inst].intersection(top3_cls)
    if len(del_set) > 0:
        for cls in del_set:
            inst2type[inst].remove(cls)

675


100%|██████████| 7204396/7204396 [00:05<00:00, 1215826.20it/s]


In [15]:
# check if empty set exists
print(len(inst2type))
clean_inst2type = {}
for inst in tqdm(inst2type.keys()):
    if len(inst2type[inst]) > 0:
        clean_inst2type[inst] = inst2type[inst].copy()
# delete inst2type
del inst2type
print(len(clean_inst2type))

7204396


100%|██████████| 7204396/7204396 [00:22<00:00, 318723.28it/s] 

5019756


In [148]:
clean_inst2type['wd:Q1258']

{'wd:Q79529'}

In [16]:
cls_insts = defaultdict(set)
for inst in tqdm(clean_inst2type.keys()):
    for cls in clean_inst2type[inst]:
        cls_insts[cls].add(inst)

100%|██████████| 5019756/5019756 [00:05<00:00, 889845.02it/s]


In [185]:
len(cls_insts)

13633

In [17]:
# The cumulative stats is for instances of classes
def getSuperClasses(cls, classes, WikiTaxonomyUp):
    """Adds all superclasses of a class <cls> (including <cls>) to the set <classes>"""
    classes.add(cls)
    # Make a check before because it's a defaultdict,
    # which would create cls if it's not there
    if cls in WikiTaxonomyUp:
        for sc in WikiTaxonomyUp[cls]:
            getSuperClasses(sc, classes, WikiTaxonomyUp)      

def getAncestors(cls, WikiTaxonomyUp):
    """Returns the set of all parent classes of <cls> (including <cls>!)"""
    classes=set()
    getSuperClasses(cls, classes, WikiTaxonomyUp)        
    return classes

def cumulative_stats(oristats, topTaxonomyUp):
    """Cumulative statistics of classes"""
    cum_stats = defaultdict(int)
    for instantiated_cls in oristats.keys():
        ancestors = getAncestors(instantiated_cls, topTaxonomyUp) # including cls itself
        for ancestor in ancestors:
            cum_stats[ancestor] += oristats[instantiated_cls]
    return cum_stats

In [21]:
# check cumulative stats
from collections import defaultdict
cls_stats = defaultdict(int)
for cls in cls_insts.keys():
    cls_stats[cls] = len(cls_insts[cls])

cum_cls_insts = cumulative_stats(cls_stats, wikiTaxonUp)

In [23]:
if 'wd:Q215627' in cum_cls_insts:
    print(cum_cls_insts['wd:Q215627'])

2575434


In [193]:
# step3: limit each class less than 1000 instances
import random
for cls in cls_insts:
    if len(cls_insts[cls]) > 100:
        sampled_insts = random.sample(cls_insts[cls], 100)
        cls_insts[cls] = set(sampled_insts).copy()

In [194]:
inst_cls = defaultdict(set)
for cls in tqdm(cls_insts):
    for inst in cls_insts[cls]:
        inst_cls[inst].add(cls)
print(len(inst_cls))

100%|██████████| 13633/13633 [00:00<00:00, 50328.12it/s]

348083


In [39]:
def load_label(path):
    cls2label = {}
    with open(path, 'r') as f:
        for line in f:
            # wd:Q96196524 rdfs:label "current entity" .
            triple = line.strip().split('\t')
            if len(triple) > 3:
                cls2label[triple[0]] = triple[2][1:-1]
    return cls2label

cls2label = load_label('/home/infres/ypeng-21/work/Taxon_clean/build_dataset/test/build_data_from_nt/24_03_22/raw_data/wikidata_src/wiki_taxonomy_labels.tsv')

* step4: select at random 100k instances

In [208]:
import random
from collections import defaultdict

sample_keys = random.sample(list(inst_cls.keys()), 50000)
sampled_inst2type = {key: inst_cls[key] for key in sample_keys}
print(len(sampled_inst2type))

ori_inst2type = defaultdict(set)
with open('Wiki_ET_subset.txt', 'r') as etreader:
    for line in etreader:
        inst, rel, cls = line.strip().split('\t')
        if inst in sampled_inst2type:
            ori_inst2type[inst].add(cls)
print(len(ori_inst2type))

50000
50000


In [209]:
# remove transtive parents types for each instance for ori
for inst in tqdm(ori_inst2type.keys()):
    if len(ori_inst2type[inst]) == 1:
        continue
    valcls = ori_inst2type[inst].copy()
    for (node1,node2) in permutations(list(ori_inst2type[inst]), 2):
        if node1 not in valcls or node2 not in valcls:
            continue
        if node1 == 'wd:Q35120': # not retyped to 'entity'
            valcls.remove(node1)
            continue
        if nx.has_path(oriWiki, node1, node2): # !!!!clean or Ori?? 不用clean的原因为，retyped到的很高层次的type很可能是错的
            valcls.remove(node1)
    ori_inst2type[inst] = valcls.copy() # set

100%|██████████| 50000/50000 [00:05<00:00, 9943.69it/s] 


In [22]:
ent2label = {}
ent2desc = {}
with open('/home/infres/ypeng-21/work/Taxon_clean/build_dataset/test/build_data_from_nt/24_03_22/raw_data/dataset/Wiki_literals.txt', 'r') as file:
    for line in file:
        qid, rel, literal = line.strip().split('\t')
        if rel == Prefixes.rdfsLabel:
            ent2label[qid] = literal[1:-4]
            continue
        if rel == Prefixes.schemaDescription:
            ent2desc[qid] = literal[1:-4]

In [210]:
import networkx as nx
from collections import deque, defaultdict

def get_parents_with_hops(graph, node):
    # node: qids
    # dictionary to store parents of node and their hop count
    parents_with_hops = defaultdict(list) # {hop_count: [parents]}
    # queue to perform BFS
    queue = deque([(node, 0)])
    visited = set()
    
    while queue:
        current_node, hop_count = queue.popleft()
        
        if current_node not in visited:
            visited.add(current_node)
            
            # For each predecessor (parent) of the current node
            for parent in graph.predecessors(current_node):
                if parent not in visited:
                    parents_with_hops[hop_count + 1].append(parent)
                    queue.append((parent, hop_count + 1))
    
    return parents_with_hops


In [223]:
import networkx as nx

def get_parents_with_depth(graph, node, cls_depth):
    # Retrieve the distances for all superclasses of the given node
    superclasses_with_distances = {}
    def collect_superclasses(node):
        for predecessor in graph.predecessors(node):
            if predecessor in cls_depth:
                superclasses_with_distances[predecessor] = cls_depth[predecessor]
                collect_superclasses(predecessor)
    
    collect_superclasses(node)
    
    return superclasses_with_distances


* write to file (as a eval dataset)

In [226]:
cls_depth = nx.single_source_shortest_path_length(cleanWiki, 'wd:Q35120')
with open('cleanWiki_eval.txt', 'w') as writer:
    for inst in tqdm(sampled_inst2type):
        types = sampled_inst2type[inst]
        parents_depth = {}
        for cls in types:
            parents_depth[cls] = cls_depth[cls]
            pars_with_dists = get_parents_with_depth(cleanWiki, cls, cls_depth)
            parents_depth.update(pars_with_dists)
        
        for cls, depth in parents_depth.items():
            writer.write(f"{inst}\t'{ent2label[inst]}'\t'{ent2desc[inst]}'\t{cls}\t{depth}\n")

  0%|          | 0/50000 [00:00<?, ?it/s]

100%|██████████| 50000/50000 [00:01<00:00, 42067.46it/s]


In [229]:
if 'wd:Q31' in sampled_inst2type:
    print(sampled_inst2type['wd:Q31'])
    print(ori_inst2type['wd:Q31'])

{'wd:Q3624078', 'wd:Q43702', 'wd:Q185441'}
{'wd:Q113489728', 'wd:Q3624078', 'wd:Q43702', 'wd:Q185441'}


In [231]:
ori_cls_depth = nx.single_source_shortest_path_length(oriWiki, 'wd:Q35120')
with open('oriWiki_eval.txt', 'w') as writer:
    for inst in tqdm(ori_inst2type):
        types = ori_inst2type[inst]
        parents_depth = {}
        for cls in types:
            parents_depth[cls] = ori_cls_depth[cls]
            pars_with_dists = get_parents_with_depth(oriWiki, cls, ori_cls_depth)
            parents_depth.update(pars_with_dists)
        
        for cls, depth in parents_depth.items():
            writer.write(f"{inst}\t'{ent2label[inst]}'\t'{ent2desc[inst]}'\t{cls}\t{depth}\n")

100%|██████████| 50000/50000 [00:10<00:00, 4778.87it/s]


* sample only 5k samples as a test! -> additional

In [232]:
import random
from collections import defaultdict

sample_keys = random.sample(list(inst_cls.keys()), 1000)
sampled_inst2type = {key: inst_cls[key] for key in sample_keys}
print(len(sampled_inst2type))

ori_inst2type = defaultdict(set)
with open('Wiki_ET_subset.txt', 'r') as etreader:
    for line in etreader:
        inst, rel, cls = line.strip().split('\t')
        if inst in sampled_inst2type:
            ori_inst2type[inst].add(cls)
print(len(ori_inst2type))

1000
1000


In [233]:
# remove transtive parents types for each instance for ori
for inst in tqdm(ori_inst2type.keys()):
    if len(ori_inst2type[inst]) == 1:
        continue
    valcls = ori_inst2type[inst].copy()
    for (node1,node2) in permutations(list(ori_inst2type[inst]), 2):
        if node1 not in valcls or node2 not in valcls:
            continue
        if node1 == 'wd:Q35120': # not retyped to 'entity'
            valcls.remove(node1)
            continue
        if nx.has_path(oriWiki, node1, node2): # !!!!clean or Ori?? 不用clean的原因为，retyped到的很高层次的type很可能是错的
            valcls.remove(node1)
    ori_inst2type[inst] = valcls.copy() # set

100%|██████████| 1000/1000 [00:00<00:00, 10656.69it/s]


In [234]:
cls_depth = nx.single_source_shortest_path_length(cleanWiki, 'wd:Q35120')
with open('cleanWiki_eval_1k.txt', 'w') as writer:
    for inst in tqdm(sampled_inst2type):
        types = sampled_inst2type[inst]
        parents_depth = {}
        for cls in types:
            parents_depth[cls] = cls_depth[cls]
            pars_with_dists = get_parents_with_depth(cleanWiki, cls, cls_depth)
            parents_depth.update(pars_with_dists)
        
        for cls, depth in parents_depth.items():
            writer.write(f"{inst}\t'{ent2label[inst]}'\t'{ent2desc[inst]}'\t{cls}\t{depth}\n")



ori_cls_depth = nx.single_source_shortest_path_length(oriWiki, 'wd:Q35120')
with open('oriWiki_eval_1k.txt', 'w') as writer:
    for inst in tqdm(ori_inst2type):
        types = ori_inst2type[inst]
        parents_depth = {}
        for cls in types:
            parents_depth[cls] = ori_cls_depth[cls]
            pars_with_dists = get_parents_with_depth(oriWiki, cls, ori_cls_depth)
            parents_depth.update(pars_with_dists)
        
        for cls, depth in parents_depth.items():
            writer.write(f"{inst}\t'{ent2label[inst]}'\t'{ent2desc[inst]}'\t{cls}\t{depth}\n")

100%|██████████| 1000/1000 [00:00<00:00, 4145.64it/s]


### Split Files

In [237]:
def split_file(input_file, output_prefix, rows_per_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    num_files = len(lines) // rows_per_file + (1 if len(lines) % rows_per_file != 0 else 0)

    for i in range(num_files):
        start_idx = i * rows_per_file
        end_idx = min((i + 1) * rows_per_file, len(lines))
        output_file = f"{output_prefix}_{i + 1}.txt"
        with open(output_file, 'w') as f:
            f.writelines(lines[start_idx:end_idx])

In [239]:
input_file = 'oriWiki_eval.txt'
output_prefix = 'oriWiki_eval'
rows_per_file = 210000
split_file(input_file, output_prefix, rows_per_file)